In [23]:
import functions as func
import numpy as np
import os
import shutil

import subprocess as sp
sampling_rate = 16000  # 采样率
file_path = '201410112308001.MP3'

# 这里直接使用命令行调用ffmpeg 大文件read_audio_file处理不了
command = [
    'ffmpeg',
    '-i', file_path,
    '-f', 's16le',  # 数据类型
    '-acodec', 'pcm_s16le',  # 解码器
    '-ar', str(sampling_rate),
    '-ac', '1',  # 单通道
    '-'  # 直接打印内容到控制台
]
# 开另一个进程跑ffmpeg
p = sp.Popen(command, stdout=sp.PIPE, bufsize=4096, close_fds=True)


# 这里拆分为10min一个文件
seconds = 10 * 60

if os.path.exists('split'):
    shutil.rmtree('split')  # 递归删除文件夹及内容
os.mkdir('split')
index = 0
while True:
    print(str(index), end='\r')  # 简略进度展示
    data = p.stdout.read(sampling_rate * 2 * seconds)  # 流式获取 不必等待文件完全加载
    if len(data) == 0:
        break
    audio_data = np.frombuffer(data, dtype=np.int16)  # 转为numpy类型
    audio_data = audio_data.astype(np.float) / np.iinfo(np.int16).max  # 转为[-1, 1]
    func.save_audio_file(r'split\{:04d}.wav'.format(index), audio_data, sampling_rate)
    index += 1
print('\nfinish')


358
finish
